<h2>FT001 - Extracción ingresos y pasivos </h2>
<p> En este apartado se trata la base de datos FT001 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [2]:
# Se carga el FT001 completo desde el archivo .csv
FT001 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/2_Mas_reciente_25082021/FT001.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object}, encoding='latin-1')

In [3]:
# Se genera una nueva columna extrayendo el mes y el año de la variable fecha
FT001['ano'] = FT001['Fecha'].str[:4]
FT001['mes'] = FT001['Fecha'].str[4:6]

In [4]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001.rename(columns={'Nit': 'nit_eps', 'codigoConcepto': 'codigo','claseConcepto': 'clase'}, inplace=True)

In [5]:
# Para hacer la suma de Valor por NIT EPS, Código del concepto, Mes y Año
FT001 = FT001.groupby(['nit_eps', 'codigo', 'ano', 'mes']).agg({'valor': 'sum'}).reset_index()

In [6]:
#Se transforman las variables a STR para mejorar su manipulación
FT001[['codigo']] = FT001[['codigo']].astype(float).astype(int).astype(str)

In [7]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT001['Lcod'] = FT001.codigo.str.len()

In [8]:
# Se limpia la clasificación por Tipo de deudor
FT001['tipo']= 'NA' # Se crea la columna con categoria UPC como base

# Se establecen las condiciones para establecer correctamente el UPC

FT001.loc[(FT001['codigo'] == "410201") |
       (FT001['codigo'] == "410202") |
       (FT001['codigo'] == "410203") |
       (FT001['codigo'] == "410207") |
       (FT001['codigo'] == "431101") |
       (FT001['codigo'] == "431102") |
       (FT001['codigo'] == "431106") |
       (FT001['codigo'] == "431122") |
       (FT001['codigo'] == "431120"),  
       'tipo'] = 'UPC' 

FT001.loc[(FT001['codigo'] == '431103') | # | Regimen contributivo
       (FT001['codigo'] == "431104") | #Regimen contributivo
       (FT001['codigo'] == "431107") | #Regimen ubsidiado publico
       (FT001['codigo'] == "41020801") | #410208-41020801
       (FT001['codigo'] == "410209"),  
       'tipo'] = 'CMCOP' 

FT001.loc[(FT001['codigo'] == '410227'),  
       'tipo'] = 'PMAX' 

FT001.loc[(FT001['codigo'] == '410230'),  
       'tipo'] = 'EnfH'

FT001.loc[(FT001['codigo'] == '1105') |
       (FT001['codigo'] == "110101") |
       (FT001['codigo'] == "110102") |
       (FT001['codigo'] == "110103") |
       (FT001['codigo'] == "110109"),  
       'tipo'] = 'Efectivo' 

In [9]:
# Para hacer la suma de Valor por NIT EPS, tipo, Mes y Año
FT001 = FT001.groupby(['nit_eps', 'tipo', 'mes', 'ano']).agg({'valor': 'sum'}).reset_index()

In [10]:
#Se transforman las variables a INT para mejorar su manipulación
FT001[['mes', 'ano']] = FT001[['mes', 'ano']].astype(float).astype(int)

<p> Desde aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [11]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT001_FECHA = FT001[
    (FT001['ano'] == 2021)
    ]

In [12]:
# Se tranforma el formato de las columnas del dataframe 
FT001_FECHA[['ano', 'mes']] = FT001_FECHA[['ano', 'mes']].astype(float).astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
# Se procede a eliminar los duplicados dejando el último movimiento registrado. Esto se debe ya que se busca trabajar con el último dato disponible
FT001_FECHA = FT001_FECHA.sort_values(['ano', 'mes']).drop_duplicates(['nit_eps'], keep='last')

In [14]:
# Para guardar la base de datos lista para ser tratada en formato CSV
FT001_FECHA.to_excel(r'C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Jul 6 Pagos WEB/Pagos WEB/EPS/2021/UNION_BASES/FT001_FECHA.xlsx', index = False)

<p> Hasta aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [15]:
%%time
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT001 = FT001.set_index('mes').groupby(['nit_eps', 'tipo', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipo', 'ano'], 1)

Wall time: 574 ms


In [16]:
# Se desagrupa la información
FT001 = FT001.reset_index()

In [17]:
#Se transforman las variables a INT para mejorar su manipulación
FT001[['ano']] = FT001[['ano']].astype(float).astype(int)

In [18]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT001[
    (FT001['ano'] == 2019) & (FT001['mes'] == 1) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 2) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 3) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 4) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 5) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 6) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 7) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 8) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 9) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 10) |
    (FT001['ano'] == 2019) & (FT001['mes'] == 11)
    ].index

In [19]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT001 = FT001.drop(df_filtered)

<p>Importante modificar la ultima línea del siguiente código "fecha_ideal" ya que aquí se define el rango en el cual se quieren tener las observaciones, en este caso está programado desde diciembre de 2019 hasta agosto de 2021. </p>

In [20]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT001[['ano', 'mes']] = FT001[['ano', 'mes']].astype(float)
FT001['periodo'] = pd.to_datetime(FT001.ano*10000 + FT001.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','8-01-2021',freq='M')

In [21]:
# Se agrupa la información según NIT EPS, tipo y periodo sumando el valor de CXC que corresponda a cada agrupación descrita
FT001 = FT001.groupby(['nit_eps', 'tipo', pd.Grouper(key='periodo', freq='M')])[['valor']].sum().reset_index()

In [22]:
# Se prepara el dataframe para calcular la diferencia periodo a periodo (t+1 - t) respetando la agregación anterior
FT001 = FT001.set_index('periodo').groupby(['nit_eps', 'tipo']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipo'], 1).reset_index()

In [23]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001.rename(columns={'level_2': 'periodo'}, inplace=True)

<p> Desde aquí se filtra la base por la fecha máxima de reporte de la información </p>

In [24]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
# FT001_FECHA = FT001_FECHA[['nit_eps', 'tipo', 'mes', 'ano']]

In [25]:
# FT001.info()

In [26]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
# FT001_FECHA.rename(columns={'mes': 'ULTIMO_MES_FT001', 'ano': 'ULTIMO_ANO_FT001'}, inplace=True)

In [27]:
# # Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
# FT001 = pd.merge(FT001, FT001_FECHA, how='left', left_on=['nit_eps', 'tipo'], right_on=['nit_eps', 'tipo'])

In [28]:
# #  Se extrae el mes y el año para porterior left join con el agregado
# FT001['ano'] = FT001['periodo'].dt.year
# FT001['mes'] = FT001['periodo'].dt.month

# #Se transforman las variables a INT para mejorar su manipulación
# FT001[['ano', 'mes']] = FT001[['ano', 'mes']].astype(int)

# # Se eliminan las columnas que no son de interés
# FT001 = FT001.drop(['periodo'], axis=1)

In [29]:
# Se crea una variable de control para filtrar por fecha del reporte
# FT001['periodo'] = pd.to_datetime(FT001.ano*10000 + FT001.mes*100 + 1, format='%Y%m%d')
# FT001['periodo_filtro'] = pd.to_datetime(FT001.ULTIMO_ANO_FT001*10000 + FT001.ULTIMO_MES_FT001*100 + 1, format='%Y%m%d')

In [30]:
# # Se filtra el dataframe con las observaciones que cumplen las condiciones. La idea es eliminar aquellas observaciones que están por fuera de las fechas máximas de reporte de cada EPS, es decir, si una EPS reportó su información 
# FT001 = FT001[
#     (FT001['periodo'] <= FT001['periodo_filtro'])
#     ]

In [31]:
# Se establece el index (identificación) como NIT EPS, tipo y el periodo para cada observación
FT001_dif = FT001.set_index(['nit_eps', 'tipo', 'periodo'])

In [32]:
# FT001_dif.columns

In [33]:
# FT001_dif = FT001_dif[['valor']]

In [34]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas (t+1 - t) y se llenan los espacios vacíos con ceros
FT001_dif = FT001_dif.diff().fillna(0).reset_index()

In [35]:
#  Se extrae el mes y el año para porterior left join con el agregado
FT001_dif['ano'] = FT001_dif['periodo'].dt.year
FT001_dif['mes'] = FT001_dif['periodo'].dt.month

FT001['ano'] = FT001['periodo'].dt.year
FT001['mes'] = FT001['periodo'].dt.month

#Se transforman las variables a INT para mejorar su manipulación
FT001[['ano', 'mes']] = FT001[['ano', 'mes']].astype(int)
FT001_dif[['ano', 'mes']] = FT001_dif[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT001 = FT001.drop(['periodo'], axis=1)
FT001_dif = FT001_dif.drop(['periodo'], axis=1)

In [36]:
# Se reemplazan los valores para los meses de enero
FT001_dif.loc[(FT001_dif['mes']) == 1 & (FT001_dif['tipo'] != 'Efectivo')] = FT001

In [37]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001_dif.rename(columns={'valor': 'Dvalormes'}, inplace=True)

In [38]:
# Se encuentran las observaciones que cumplen las condiciones (se eliminan las observaciones de 2019)
df_filtered = FT001_dif[
    (FT001_dif['ano'] == 2019)
    ].index
# Se eliminan las observaciones que cumplen las condiciones anteriores (se eliminan las observaciones de 2019)
FT001_dif = FT001_dif.drop(df_filtered)

In [39]:
# Para convertir el tipo como columna para la base de datos con la diferencia de un periodo
FT001_dif = FT001_dif.pivot(index=['nit_eps', 'mes', 'ano'], columns='tipo', values='Dvalormes').fillna(0).reset_index()

In [40]:
# Para convertir el tipo como columna para la base de datos agregada
FT001 = FT001.pivot(index=['nit_eps', 'mes', 'ano'], columns='tipo', values='valor').fillna(0).reset_index()

In [41]:
# Se eliminan las columnas que no son de interés en la base no diferenciada
FT001 = FT001.drop(['CMCOP', 'EnfH', 'NA', 'PMAX', 'UPC'], axis=1)

In [42]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento en la base no diferenciada
FT001.rename(columns={'Efectivo': 'valorEfectivo'}, inplace=True)

In [43]:
# Se unen las dos bases de datos para obtener la información en conjunto de la diferencia y el valor de Efectivo
FT001_dif = pd.merge(FT001_dif, FT001, how='left', left_on=['nit_eps', 'ano', 'mes'], right_on=['nit_eps', 'ano', 'mes'])

In [44]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT001_dif.rename(columns={'Efectivo': 'DEfectivo', 'CMCOP': 'DCMCOPmes', 'PMAX': 'DPMAXmes', 'UPC': 'DUPCmes'}, inplace=True)

<p> Desde aquí se filtra la base por la fecha máxima de reporte de la información </p>

In [45]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
FT001_FECHA = FT001_FECHA[['nit_eps', 'mes', 'ano']]

In [46]:
FT001_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1045 entries, 0 to 1044
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nit_eps        1045 non-null   object 
 1   mes            1045 non-null   int64  
 2   ano            1045 non-null   int64  
 3   DCMCOPmes      1045 non-null   float64
 4   DEfectivo      1045 non-null   float64
 5   EnfH           1045 non-null   float64
 6   NA             1045 non-null   float64
 7   DPMAXmes       1045 non-null   float64
 8   DUPCmes        1045 non-null   float64
 9   valorEfectivo  1045 non-null   float64
dtypes: float64(7), int64(2), object(1)
memory usage: 89.8+ KB


In [47]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
FT001_FECHA.rename(columns={'mes': 'ULTIMO_MES_FT001', 'ano': 'ULTIMO_ANO_FT001'}, inplace=True)

In [48]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT001 = pd.merge(FT001_dif, FT001_FECHA, how='left', left_on=['nit_eps'], right_on=['nit_eps'])

In [49]:
# Se crea una variable de control para filtrar por fecha del reporte
FT001['periodo'] = pd.to_datetime(FT001.ano*10000 + FT001.mes*100 + 1, format='%Y%m%d')
FT001['periodo_filtro'] = pd.to_datetime(FT001.ULTIMO_ANO_FT001*10000 + FT001.ULTIMO_MES_FT001*100 + 1, format='%Y%m%d')

In [52]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones. La idea es eliminar aquellas observaciones que están por fuera de las fechas máximas de reporte de cada EPS, es decir, si una EPS reportó su información 
FT001 = FT001[
    (FT001['periodo'] <= FT001['periodo_filtro'])
    ]

In [53]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT001_dif.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/ingresosyefectivo_2020_ene_may_2021.xlsx', index = False)
FT001.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/ingresosyefectivo_2020_ene_jul_2021.csv', index = False, encoding='utf-8-sig')

In [54]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT001_dif, FT001, FT001_FECHA, df_filtered, fecha_ideal, df]